In [10]:
import sys
import pandas as pd

from os.path import join
from tqdm import tqdm

#from src.strategy import TimeBasedRule

# From ../src directory and utils module import get_texas_season
# Add path for importing the get_texas_season function

sys.path.append('../src/')
from utils import get_texas_season
from strategy import TimeBasedRule

In [11]:
DATA_BASE_PATH = '../data/'

In [12]:
virtual_trading_dataset = pd.read_csv(join(DATA_BASE_PATH, 'virtual_trading_data.csv'))
virtual_trading_dataset["date"] = pd.to_datetime(virtual_trading_dataset["date"])

In [13]:
virtual_trading_dataset["day_of_week"] = virtual_trading_dataset["date"].dt.day_name()
virtual_trading_dataset["hour"] = virtual_trading_dataset["date"].dt.hour

In [14]:
node_list = virtual_trading_dataset['settlementPoint'].unique()

In [15]:
list_calcs_node = []
for node in tqdm(node_list):
    node_data = virtual_trading_dataset[virtual_trading_dataset['settlementPoint'] == node].copy()
    mu_returns = node_data['return_DA_RT'].mean()
    sigma_returns = node_data['return_DA_RT'].std()
    sharpe_ratio_node = mu_returns / sigma_returns
    # Do not consider too high margins
    # node_data = node_data[node_data['return_DA_RT'] < 500]
    node_data["n_hours_trading"] = 1
    node_data["wins"] = (node_data["return_DA_RT"] > 0).astype(int)
    node_data = node_data.groupby(['day_of_week', 'hour']).agg({
        'return_DA_RT': ('sum', 'std', 'mean', pd.Series.skew, pd.Series.kurt, 'median'),
        'n_hours_trading': 'sum',
        'wins': 'sum',
        'SPP_DA': 'mean',
        'SPP_RT': 'mean'
    }).reset_index()
    node_data["start_date"] = virtual_trading_dataset["date"].min()
    node_data["end_date"] = virtual_trading_dataset["date"].max()
    node_data["settlementPoint"] = node
    node_data["sharpe_ratio"] = node_data["return_DA_RT"]["mean"] / node_data["return_DA_RT"]["std"]
    # Calculate the sharpe ratio for the node
    node_data["node_sharpe_ratio"] = sharpe_ratio_node
    node_data["node_win_rate"] = node_data["wins"]["sum"].sum() / node_data["n_hours_trading"]["sum"].sum()
    list_calcs_node.append(node_data)

100%|██████████| 800/800 [04:15<00:00,  3.13it/s]


In [16]:
complete_df_sorted = pd.concat(list_calcs_node).sort_values(by=[('n_hours_trading', 'sum'), ('return_DA_RT', 'median'), ('return_DA_RT', 'std')], ascending=[False, False, True])
complete_df_sorted["win_rate"] = complete_df_sorted["wins"]["sum"] / complete_df_sorted["n_hours_trading"]["sum"]

In [17]:
complete_df_sorted

day_of_week hour return_DA_RT                                              \
                              sum         std       mean      skew       kurt   
45       Monday   21     185.2925   52.593129   4.309128 -0.118659   2.381123   
46       Monday   22     108.6325   28.712582   2.414056 -2.236492   9.803652   
46       Monday   22     101.4500   27.923984   2.254444 -2.385844  10.993639   
46       Monday   22     -57.3900   51.499382  -1.334651 -0.812464   2.627888   
45       Monday   21      96.0200   32.247653   2.133778 -0.871330   2.933237   
..          ...  ...          ...         ...        ...       ...        ...   
114    Thursday   18    -165.7675   72.435404 -33.153500 -0.044652  -0.614314   
113    Thursday   17      80.4825  107.359921  16.096500  1.847234   3.884884   
162   Wednesday   18    -182.4425   25.925143 -36.488500 -1.188197   0.678201   
163   Wednesday   19    -148.5600   14.096753 -29.712000  0.068657  -1.218200   
115    Thursday   19    -358.5850   84.527821 -71.717000 -0.720350   1.798429   

             n_hours_trading wins      SPP_DA     SPP_RT          start_date  \
      median             sum  sum        mean       mean                       
45    5.2625              49   26   30.610408  36.478198 2023-12-11 14:00:00   
46    5.0200              49   29    8.025918  12.209167 2023-12-11 14:00:00   
46    4.9750              49   29    8.649796  12.728889 2023-12-11 14:00:00   
46    4.4450              49   24   26.092449  26.813721 2023-12-11 14:00:00   
45    4.2875              49   28   13.512857  17.578444 2023-12-11 14:00:00   
..       ...             ...  ...         ...        ...                 ...   
114 -19.6700               9    1   90.376667  64.388500 2023-12-11 14:00:00   
113 -20.6850               9    1   57.545556  76.326500 2023-12-11 14:00:00   
162 -25.5000               9    0  109.554444  37.309500 2023-12-11 14:00:00   
163 -28.6400               9    0   84.132222  49.472000 2023-12-11 14:00:00   
115 -64.1300               9    1   75.738889  16.229000 2023-12-11 14:00:00   

               end_date settlementPoint sharpe_ratio node_sharpe_ratio  \
                                                                         
45  2024-11-12 14:00:00     MASSENGL_G8     0.081933         -0.026476   
46  2024-11-12 14:00:00            AEEC     0.084077         -0.023229   
46  2024-11-12 14:00:00     PALE_ESS_EN     0.080735         -0.023389   
46  2024-11-12 14:00:00     MASSENGL_G8    -0.025916         -0.026476   
45  2024-11-12 14:00:00     PALE_ESS_EN     0.066168         -0.023389   
..                  ...             ...          ...               ...   
114 2024-11-12 14:00:00       FORTMA_RN    -0.457697         -0.093566   
113 2024-11-12 14:00:00       FORTMA_RN     0.149930         -0.093566   
162 2024-11-12 14:00:00       FORTMA_RN    -1.407456         -0.093566   
163 2024-11-12 14:00:00       FORTMA_RN    -2.107719         -0.093566   
115 2024-11-12 14:00:00       FORTMA_RN    -0.848443         -0.093566   

    node_win_rate  win_rate  
                             
45       0.403561  0.530612  
46       0.449926  0.591837  
46       0.448071  0.591837  
46       0.403561  0.489796  
45       0.448071  0.571429  
..            ...       ...  
114      0.245841  0.111111  
113      0.245841  0.111111  
162      0.245841  0.000000  
163      0.245841  0.000000  
115      0.245841  0.111111  

[134400 rows x 19 columns]

In [24]:
TEST_NODE = 'LZ_NORTH'

In [25]:
test_node_data = complete_df_sorted[complete_df_sorted['settlementPoint'] == TEST_NODE].copy()

In [26]:
test_node_data.pivot(
    columns='day_of_week',
    index='hour',
    values=('return_DA_RT', 'median')
).style.background_gradient(cmap='hot', axis=None, vmin=0, vmax=10).format("{:.2f}")

day_of_week,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
hour,,,,,,,
0,-0.19,0.08,0.51,0.66,-0.16,0.72,-0.64
1,0.85,0.39,1.43,0.93,0.47,1.31,-0.09
2,0.80,0.23,1.83,0.69,0.72,1.73,-0.61
3,-0.10,0.63,1.30,1.06,0.62,1.68,-0.13
4,-0.03,0.85,0.85,1.10,-0.57,1.37,-0.70
5,-0.44,0.97,0.86,-0.39,-0.17,0.81,-0.88
6,-1.00,1.02,-0.44,-0.08,1.20,0.55,-0.60
7,-0.41,0.80,-0.43,0.42,0.62,0.13,-1.52
8,-0.64,-0.90,-0.50,-1.19,-0.62,-1.32,-1.64


In [27]:
def generate_rules(cond):
    df_node_weekly = test_node_data.sort_values(by=["day_of_week", "hour"])
    rules = []
    for dow in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]:
        h_list = []
        for hour in range(24):
            df_local = df_node_weekly[(df_node_weekly["day_of_week"] == dow) & (df_node_weekly["hour"] == hour)]
            if cond(df_local["return_DA_RT"]["median"].values[0]):
                h_list.append(hour)

        if len(h_list) > 0:
            rules.append(TimeBasedRule(day_of_week=dow, hour_range=h_list))

    return rules

In [28]:
rules = generate_rules(lambda x: x > 1)

In [29]:
for rule in rules:
    dow = rule.day_of_week
    tr = rule.hour_range
    print(f"TimeBasedRule(day_of_week='{dow}', hour_range={tr}),")

TimeBasedRule(day_of_week='Monday', hour_range=[6, 14]),
TimeBasedRule(day_of_week='Tuesday', hour_range=[1, 2, 3, 4]),
TimeBasedRule(day_of_week='Thursday', hour_range=[6]),
TimeBasedRule(day_of_week='Saturday', hour_range=[1, 2, 3, 23]),
TimeBasedRule(day_of_week='Sunday', hour_range=[3, 4, 23]),


In [33]:
# Concatenate the second level of the index
complete_df_sorted.columns = ['_'.join(col).strip() for col in complete_df_sorted.columns.values]

In [35]:
complete_df_sorted.to_csv(join(DATA_BASE_PATH, 'virtual_trading_weekday_hour.csv'), index=False)